In [1]:
%cd /content/drive/MyDrive/_main_/1_FL/data_energy/datalog

/content/drive/MyDrive/_main_/1_FL/data_energy/datalog


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf ### AI
import sklearn

import json
from glob import glob

In [3]:
def get_data(lista_j):
    datas_x1 = pd.DataFrame(lista_j)
    datas_x2 = pd.DataFrame([i['data'] for i in lista_j])
    datas = pd.concat([datas_x1, datas_x2], axis=1).drop(columns=['data'])
    datas['fechaCreacion'] = datas['fechaCreacion'].astype('datetime64[ms]')
    datas = datas.sort_values('fechaCreacion')
    datas.index = datas['fechaCreacion']
    # datas = datas.resample('1min').sum()
    datas = datas.resample('1min').mean()
    # return datas.drop(columns=['idLog', 'idSensor'])[['corriente', 'voltaje', 'potencia', 'potencia_efectiva', 'energia', 'frecuencia', 'pf', 'fechaCreacion']]
    # return datas[['corriente', 'voltaje', 'potencia', 'potencia_efectiva', 'energia', 'frecuencia', 'pf', 'fechaCreacion']]
    # return datas[['corriente', 'voltaje', 'potencia', 'potencia_efectiva', 'energia', 'frecuencia', 'pf']]
    return datas[['corriente', 'potencia', 'potencia_efectiva', 'pf']]

In [4]:
def get_vector(vector_x):
    # dx = dataframe.iloc[15:29] ### 14 values
    dx = vector_x.copy()
    n = dx.shape[0]
    X = []
    for i in range(0,n, 2):
        j = i+2
        # print(i, j)
        dy = dx.iloc[i:j].values.mean(axis=0).tolist()
        X.append(dy)
        # print(dy)

    xx = np.array(X).flatten().tolist()
    return xx

In [5]:
def get_XT(dataf):
    
    XT = []
    n = dataf.shape[0] ### size del dataframe
    for i in range(0, n-14): ### iteracion del dataframe | 14 frames
        m = i+14
        x_0 = dataf.iloc[i:m]
        nas = x_0.isna().values.sum()
        # print(nas)
        if nas==0:
            vector_x = get_vector(x_0) ### x_vector
            # print(vector_x)
            XT.append(vector_x)
    
    return np.array(XT)

def get_XT2(dataf):
    
    XT = []
    n = dataf.shape[0] ### size del dataframe
    for i in range(0, n-14): ### iteracion del dataframe | 14 frames
        m = i+14
        x_0 = dataf.iloc[i:m]
        nas = x_0.isna().values.sum()
        # print(nas)
        if nas==0:
            vector_x = get_vector(x_0) ### x_vector
            # print(vector_x)
            XT.append(vector_x)
    
    return np.array(XT)

In [ ]:
glob("*json")

['ventilador_on.json',
 'lampara_on.json',
 'congelador_on.json',
 'congelador_v1.json',
 'lampara_v1.json',
 'ventilador_v1.json']

In [6]:
### load jsons
v1 = json.load(open('ventilador_on.json'))
v2 = json.load(open('ventilador_v1.json'))
l1 = json.load(open('lampara_on.json'))
l2 = json.load(open('lampara_v1.json'))
c1 = json.load(open('congelador_on.json'))
c2 = json.load(open('congelador_v1.json'))

In [ ]:
# convolucional

In [ ]:
v1[:2]

[{'data': {'corriente': 0.44,
   'energia': 0.87,
   'frecuencia': 60.0,
   'pf': 0.97,
   'potencia': 52.711999999999996,
   'potencia_efectiva': 51.3,
   'voltaje': 119.8},
  'fechaCreacion': '2022-07-01T18:38:41.704+00:00',
  'idLog': '62bf3f31c03a423caae7c9e6',
  'idSensor': '98:CD:AC:31:CE:BF'},
 {'data': {'corriente': 0.44,
   'energia': 0.87,
   'frecuencia': 60.0,
   'pf': 0.96,
   'potencia': 52.711999999999996,
   'potencia_efectiva': 51.2,
   'voltaje': 119.8},
  'fechaCreacion': '2022-07-01T18:38:45.772+00:00',
  'idLog': '62bf3f35c03a423caae7c9e7',
  'idSensor': '98:CD:AC:31:CE:BF'}]

In [7]:
### load dataframes
dv1 = get_data(v1)
dv2 = get_data(v2)
dl1 = get_data(l1)
dl2 = get_data(l2)
dc1 = get_data(c1)
dc2 = get_data(c2)

In [8]:
dv1

,corriente,potencia,potencia_efectiva,pf
fechaCreacion,,,,
2022-07-01 18:28:00,0.44,52.858667,51.433333,0.966667
2022-07-01 18:29:00,NaN,NaN,NaN,NaN
2022-07-01 18:30:00,NaN,NaN,NaN,NaN
2022-07-01 18:31:00,NaN,NaN,NaN,NaN
2022-07-01 18:32:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2022-07-01 21:08:00,0.44,52.806286,51.450000,0.965000
2022-07-01 21:09:00,0.44,52.629867,51.080000,0.964000
2022-07-01 21:10:00,0.44,52.662133,51.160000,0.963333


In [9]:
### get X per class
v1 = get_XT(dv1)
v2 = get_XT(dv2)
l1 = get_XT(dl1)
l2 = get_XT(dl2)
c1 = get_XT(dc1)
c2 = get_XT(dc2)

### clases
dicc = {
    "ventilador":[v1, v2],
    "lampara":[l1, l2],
    "congelador":[c1, c2]
}

In [12]:
v1.shape

(144, 28)

In [13]:
data = np.array([[0]*29])
ll = 0
for i, llave in enumerate(dicc):
    lista = dicc[llave]
    for x_vector in lista:
        nm = x_vector.shape[0]
        lista_y = np.array([i]*nm)
        arr = np.concatenate((x_vector, lista_y.reshape(-1, 1)), axis=1)
        # data.append(arr)
        data = np.concatenate([data, arr], axis=0)
        ll+=1

# arr_data = np.array(data)
df = pd.DataFrame(data[1:])
df.columns = [f'feature_{i}' for i in range(1, 29)] + ['label']
df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,label
0,0.440000,52.354867,50.646667,0.964833,0.440000,52.613838,51.090714,0.964476,0.440000,52.722267,...,0.964667,0.440000,52.627457,51.109524,0.964095,0.440000,52.648933,51.143333,0.963333,0.0
1,0.440000,52.499438,50.890714,0.964476,0.440000,52.692933,51.230000,0.964000,0.440000,52.657733,...,0.964667,0.440000,52.727190,51.276190,0.963095,0.440000,52.657733,51.160000,0.964000,0.0
2,0.440000,52.613838,51.090714,0.964476,0.440000,52.722267,51.266667,0.964000,0.440000,52.622848,...,0.964095,0.440000,52.648933,51.143333,0.963333,0.440000,52.680990,51.189286,0.963048,0.0
3,0.440000,52.692933,51.230000,0.964000,0.440000,52.657733,51.160000,0.964667,0.440000,52.590581,...,0.963095,0.440000,52.657733,51.160000,0.964000,0.440000,52.679524,51.175952,0.962714,0.0
4,0.440000,52.722267,51.266667,0.964000,0.440000,52.622848,51.111190,0.964476,0.440000,52.596133,...,0.963333,0.440000,52.680990,51.189286,0.963048,0.440000,52.747200,51.303333,0.964000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,2.103238,251.247483,163.187381,0.649619,1.984000,236.821800,149.040000,0.629667,2.035643,243.450924,...,0.610000,2.014595,242.496036,146.297857,0.602333,2.018667,243.936167,145.996667,0.600000,2.0
1067,2.068000,246.760633,157.063333,0.635667,1.975310,235.996757,147.349762,0.625405,2.026000,242.177067,...,0.607333,2.018595,243.575236,146.277857,0.600000,1.160333,140.273767,82.946667,0.413667,2.0
1068,1.984000,236.821800,149.040000,0.629667,2.035643,243.450924,151.139762,0.621071,2.015000,240.525500,...,0.602333,2.018667,243.936167,145.996667,0.600000,0.171690,20.738300,10.510000,0.198310,2.0
1069,1.975310,235.996757,147.349762,0.625405,2.026000,242.177067,149.583333,0.619000,2.012143,240.565857,...,0.600000,1.160333,140.273767,82.946667,0.413667,0.039357,4.773100,0.840000,0.156643,2.0


In [14]:
ventilador = df[df.label==0]
lampara = df[df.label==1]
congelador = df[df.label==2]

In [15]:
df.groupby('label').size()

label
0.0    324
1.0    294
2.0    453
dtype: int64

In [ ]:
df_resample = pd.concat([ventilador.sample(n = lampara.shape[0], random_state = 100),
                         lampara,
                         congelador.sample(n = lampara.shape[0], random_state = 100)
                        ], axis=0)

X, y = df_resample.loc[:, df_resample.columns != 'label'], df_resample['label']
X.shape, y.shape

((882, 28), (882,))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [ ]:
from sklearn.metrics import classification_report

target_names = ['ventilador', 'lampara', 'congelador']
y_pred = clf.predict(x_test)

print(classification_report(y_pred, y_pred, target_names=target_names))

              precision    recall  f1-score   support

  ventilador       1.00      1.00      1.00        74
     lampara       1.00      1.00      1.00        77
  congelador       1.00      1.00      1.00        70

    accuracy                           1.00       221
   macro avg       1.00      1.00      1.00       221
weighted avg       1.00      1.00      1.00       221



In [ ]:
%%timeit
clf.predict(x_test)

The slowest run took 4.52 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 5: 1.93 ms per loop


In [ ]:
clf.predict(x_test[0].reshape(-1, 1).T)

array([0.])

In [ ]:
clf.predict_proba?

Object `clf.predict_proba` not found.


In [ ]:
clf.predict_proba(x_test[0].reshape(-1, 1).T)

AttributeError: ignored

In [ ]:
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

((661, 28), (661,), (221, 28), (221,))


In [ ]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input, Activation, Dropout
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
max_len = 28
max_words = 28

inputs = Input(name='inputs',shape=[max_len])

layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 28)]              0         
                                                                 
 embedding_5 (Embedding)     (None, 28, 50)            1400      
                                                                 
 lstm_5 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_9 (Activation)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, 
          epochs=10)

Epoch 1/10


InvalidArgumentError: ignored